In [6]:
import numpy as np
import pandas as pd
import requests
import re
import time
import pickle
import datetime

from bs4 import BeautifulSoup as bs


C:\Users\HSS\Documents\Anaconda3\envs\tensorflow\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
C:\Users\HSS\Documents\Anaconda3\envs\tensorflow\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [104]:
#네이버 도전웹툰 1~1000page의 모든 웹툰 제목(title), 링크(link), 평점(star)를 수집
def get_challenge_webtoon(category, page):
    url = 'https://comic.naver.com/genre/bestChallenge.nhn?m={}&page={}'.format(category, page)
    response = requests.get(url)
    webtoons = bs(response.content, 'html.parser')
    
    domain = 'https://comic.naver.com'
    for webtoon in webtoons.select('.challengeInfo'):
        table.append({
            
            'title' : webtoon.select('a')[0].text.strip(),
            'link' : domain + webtoon.select('a')[0].get('href'),
            'star' : webtoon.select_one('.star').get('title'),
            'category': category
        })

#link를 받아 에피소드의 개수를 체크함 (첫번째 url = 마지막 에피소드 url에 회차정보가 담겨 있음)       
def check_num_episode(title_link):
    response = requests.get(title_link)
    docs = bs(response.content, 'html.parser')
    last_episode_html = docs.select_one('.title > a').get('href')
    last_episode_date = docs.select_one('.num').text
#     likes = docs.select_one('em.u_cnt').text
    
    return last_episode_html[-4:], last_episode_date

In [90]:
url = 'https://comic.like.naver.com/likeIt/likeItContent.jsonp?_callback=window.__jindo2_callback.$2328&serviceId=COMIC&displayId=COMIC&contentsId=729593&lang=ko&viewType=like'
response = requests.get(url)
response

<Response [200]>

In [98]:
response

<Response [200]>

In [77]:
docs.select('div.detail > ul >li:nth-child(5) > div.u_likeit_module')[0]

<div class="u_likeit_module">
</div>

[]

In [20]:
table = []
categories = {'0': 'episode',
              '1': 'omnibus',
              '2': 'story',
              '3': 'daily',
              '4': 'comic',
              '5': 'fantasy',
              '6': 'action',
              '7': 'drama',
              '8': 'pure',
              '9': 'sensibility',
              '10': 'thrill',
              '11': 'historical',
              '12': 'sports',
             }

for i,category in categories.items():
    for page in range(1,2):
        get_challenge_webtoon(category,page)
        
result = pd.DataFrame(table)
result.star = result.star.apply(lambda x: float(x.replace('점','')))
result['title_id'] = result.link.apply(lambda x:x[-6:])
result.tail()


,category,link,star,title,title_id
307,story,https://comic.naver.com/bestChallenge/list.nhn...,9.87,숲속의 성,541226
308,story,https://comic.naver.com/bestChallenge/list.nhn...,9.59,공주님은 알바중,735988
309,story,https://comic.naver.com/bestChallenge/list.nhn...,9.89,하나밖에 안 보여!,644886
310,story,https://comic.naver.com/bestChallenge/list.nhn...,9.86,Twins Love Story (트윈스 러브 스토리),499692
311,story,https://comic.naver.com/bestChallenge/list.nhn...,9.78,심연,695484


In [ ]:
for page in range(1,2):#원래 range(1,1000)
    get_challenge_webtoon(page)

result = pd.DataFrame(table)
result.star = result.star.apply(lambda x: float(x.replace('점','')))
result['title_id'] = result.link.apply(lambda x:x[-6:])
result.tail()






In [105]:
#나중에함수
links = result.link
num = []
date = []
likes = []


for i in range(len(links)):
    
    try:
        num.append(check_num_episode(links[i])[0])
        date.append(check_num_episode(links[i])[1])
#         likes.append(check_num_episode(links[i][2]))
        
    except AttributeError:
        num.append('0')
        date.append('2000.01.01')
#         likes.append('0')
        
    if i%30 == 0:
        print(str(i+1) +'/{}'.format(len(links)))

1/312
31/312
61/312
91/312
121/312
151/312
181/312
211/312
241/312
271/312
301/312


In [106]:
result['num_episode'] = num
result['num_episode'] = result.num_episode.apply(lambda x: re.sub('[a-z=]','',x))
result['last_date'] = date
# result['likes'] = likes

now = datetime.datetime.now()
now = datetime.datetime.date(now)

result['last_date'] = result['last_date'].apply(lambda x: datetime.datetime.date(datetime.datetime.strptime(x, '%Y.%m.%d')))
result.head(2)




,category,link,star,title,title_id,num_episode,last_date,likes
0,omnibus,https://comic.naver.com/bestChallenge/list.nhn...,9.93,잔액이 부족합니다,729593,39,2020-03-10,0
1,omnibus,https://comic.naver.com/bestChallenge/list.nhn...,9.84,하루한끼 감정일기,696751,52,2020-03-09,0


In [107]:
result.head(30)

,category,link,star,title,title_id,num_episode,last_date,likes
0,omnibus,https://comic.naver.com/bestChallenge/list.nhn...,9.93,잔액이 부족합니다,729593,39,2020-03-10,0
1,omnibus,https://comic.naver.com/bestChallenge/list.nhn...,9.84,하루한끼 감정일기,696751,52,2020-03-09,0
2,omnibus,https://comic.naver.com/bestChallenge/list.nhn...,9.70,정신과심리만화 by 팔호광장,729599,79,2020-03-09,0
3,omnibus,https://comic.naver.com/bestChallenge/list.nhn...,9.89,코스믹 코믹스,663704,69,2020-03-09,0
4,omnibus,https://comic.naver.com/bestChallenge/list.nhn...,9.83,신혼좀비,735601,66,2020-03-09,0
5,omnibus,https://comic.naver.com/bestChallenge/list.nhn...,8.43,아좀비,736647,45,2020-03-10,0
6,omnibus,https://comic.naver.com/bestChallenge/list.nhn...,9.85,결혼하고 싶지 않았는데 못하게 되었다.,733288,44,2020-03-09,0
7,omnibus,https://comic.naver.com/bestChallenge/list.nhn...,9.83,우주수렴법칙,734317,29,2020-03-09,0
8,omnibus,https://comic.naver.com/bestChallenge/list.nhn...,9.22,158동 진상부부,726814,89,2020-03-09,0
9,omnibus,https://comic.naver.com/bestChallenge/list.nhn...,9.39,도와줘요 사신님!,734851,27,2020-03-09,0


In [ ]:
result.to_csv('sample.csv')